In [8]:
import sqlite3
import sys
import requests_cache
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [9]:
sys.path.insert(0, "/Users/samdillard/python/github.com/samhld/nba_stats/app/")
requests_cache.install_cache()

In [3]:
from scraper import getters
from preprocessor import builders
from PandasBasketball.stats import player_stats
from preprocessor.builders import create_player_stats_df

In [5]:
url = "http://www.basketball-reference.com/players/c/cookqu01.html"
# url = "http://www.basketball-reference.com/players/b/brookaa01.html"
stat = "per_game"
# player = "Aaron Brooks"
res = requests.get(url)
player_df = player_stats(res, stat)
level1 = [player]*len(player_df)
level2 = list(range(1,len(player_df)+1))
mi = pd.MultiIndex.from_arrays([level1, level2], names=("Player", "Year"))
player_df = player_df.set_index(mi)
soup = BeautifulSoup(res.text, "lxml")
player = soup.find("h1").text.strip()
player_df

Season Age   Tm   Lg Pos   G  GS    MP   FG  FGA  ...   FT%  \
Player     Year                                                     ...         
Quinn Cook 1     2016-17  23  TOT  NBA  PG  14   0  13.4  2.4  4.7  ...  .400   
           2     2016-17  23  DAL  NBA  PG   5   0  15.4  2.2  5.0  ...  .000   
           3     2016-17  23  NOP  NBA  PG   9   0  12.3  2.4  4.6  ...  .667   
           4     2017-18  24  GSW  NBA  PG  33  18  22.4  3.7  7.6  ...  .880   
           5     2018-19  25  GSW  NBA  PG  74  10  14.3  2.8  5.9  ...  .769   
           6     2019-20  26  LAL  NBA  PG  44   1  11.5  2.1  4.9  ...  .786   
           7     2020-21  27  LAL  NBA  PG  11   0   3.6  0.7  1.5  ...  .800   

                 ORB  DRB  TRB  AST  STL  BLK  TOV   PF  PTS  
Player     Year                                               
Quinn Cook 1     0.1  0.4  0.5  1.9  0.3  0.0  0.9  0.9  5.6  
           2     0.0  0.6  0.6  2.4  0.2  0.0  1.6  1.2  5.4  
           3     0.1  0.3  0.4  1.6  0.3  0.0  0.6  0.8  5.8  
           4     0.3  2.2  2.5  2.7  0.4  0.0  1.0  1.8  9.5  
           5     0.3  1.8  2.1  1.6  0.3  0.0  0.7  1.2  6.9  
           6     0.3  0.9  1.2  1.1  0.3  0.0  0.8  0.6  5.1  
           7     0.0  0.3  0.3  0.3  0.1  0.0  0.1  0.1  2.2  

[7 rows x 30 columns]

In [26]:
url = "http://www.basketball-reference.com/players/c/cookqu01.html"
# url = "https://www.basketball-reference.com/players/h/hardeja01.html"
stat = "per_game"
requests.get(url).text

''

In [26]:
seg_prev = slice(201,509)
mid_300ish = builders.create_all_player_stats_df(segment=seg_prev)

url: http://www.basketball-reference.com/players/b/banede01.html. ('Desmond Bane', 1)
url: http://www.basketball-reference.com/players/b/banksge01.html. ('Gene Banks', 1)
url: http://www.basketball-reference.com/players/b/banksma01.html. ('Marcus Banks', 1)
url: http://www.basketball-reference.com/players/b/bankswa01.html. ('Walker Banks', 1)
url: http://www.basketball-reference.com/players/b/bannike01.html. ('Ken Bannister', 1)
url: http://www.basketball-reference.com/players/b/bantomi01.html. ('Mike Bantom', 1)
url: http://www.basketball-reference.com/players/b/barbejo01.html. ('John Barber', 1)
url: http://www.basketball-reference.com/players/b/barbole01.html. ('Leandro Barbosa', 1)
url: http://www.basketball-reference.com/players/b/bardost01.html. ('Steve Bardo', 1)
url: http://www.basketball-reference.com/players/b/bareajo01.html. ('J.J. Barea', 1)
url: http://www.basketball-reference.com/players/b/bargnan01.html. ('Andrea Bargnani', 1)
url: http://www.basketball-reference.com/pla

url: http://www.basketball-reference.com/players/b/belljo01.html. ('Jordan Bell', 1)
url: http://www.basketball-reference.com/players/b/bellra01.html. ('Raja Bell', 1)
url: http://www.basketball-reference.com/players/b/belltr01.html. ('Troy Bell', 1)
url: http://www.basketball-reference.com/players/b/bellwh01.html. ('Whitey Bell', 1)
url: http://www.basketball-reference.com/players/b/bellawa01.html. ('Walt Bellamy', 1)
url: http://www.basketball-reference.com/players/b/bembrde01.html. ("DeAndre' Bembry", 1)
url: http://www.basketball-reference.com/players/b/bemorir01.html. ('Irv Bemoras', 1)
url: http://www.basketball-reference.com/players/b/benbole01.html. ('Leon Benbow', 1)
url: http://www.basketball-reference.com/players/b/bendedr01.html. ('Dragan Bender', 1)
url: http://www.basketball-reference.com/players/b/bendejo01.html. ('Jonathan Bender', 1)
url: http://www.basketball-reference.com/players/b/benimje01.html. ('Jerrelle Benimon', 1)
url: http://www.basketball-reference.com/playe

url: http://www.basketball-reference.com/players/b/bogdabo02.html. ('Bojan Bogdanović', 1)
url: http://www.basketball-reference.com/players/b/boguemu01.html. ('Muggsy Bogues', 1)
url: http://www.basketball-reference.com/players/b/bogutan01.html. ('Andrew Bogut', 1)
url: http://www.basketball-reference.com/players/b/bohanet01.html. ('Etdrick Bohannon', 1)
url: http://www.basketball-reference.com/players/b/bolbo01.html. ('Bol Bol', 1)
url: http://www.basketball-reference.com/players/b/bolma01.html. ('Manute Bol', 1)
url: http://www.basketball-reference.com/players/b/boldejo01.html. ('Jonah Bolden', 1)
url: http://www.basketball-reference.com/players/b/boldema01.html. ('Marques Bolden', 1)
url: http://www.basketball-reference.com/players/b/bolgebi01.html. ('Bill Bolger', 1)
url: http://www.basketball-reference.com/players/b/bolomjo01.html. ('Joel Bolomboy', 1)
url: http://www.basketball-reference.com/players/b/bolstdo01.html. ('Doug Bolstorff', 1)
url: http://www.basketball-reference.com/

url: http://www.basketball-reference.com/players/b/breweco01.html. ('Corey Brewer', 1)
url: http://www.basketball-reference.com/players/b/breweja01.html. ('Jamison Brewer', 1)
url: http://www.basketball-reference.com/players/b/breweji01.html. ('Jim Brewer', 1)
url: http://www.basketball-reference.com/players/b/brewero01.html. ('Ron Brewer', 1)
url: http://www.basketball-reference.com/players/b/brewero02.html. ('Ronnie Brewer', 1)
url: http://www.basketball-reference.com/players/b/brezepr01.html. ('Primož Brezec', 1)
url: http://www.basketball-reference.com/players/b/brianfr01.html. ('Frankie Brian', 1)
url: http://www.basketball-reference.com/players/b/brickfr01.html. ('Frank Brickowski', 1)
url: http://www.basketball-reference.com/players/b/bridgju01.html. ('Junior Bridgeman', 1)
url: http://www.basketball-reference.com/players/b/bridgbi01.html. ('Bill Bridges', 1)
url: http://www.basketball-reference.com/players/b/bridgmi01.html. ('Mikal Bridges', 1)
url: http://www.basketball-refere

In [10]:
seg = slice(510,810)
next_300 = builders.create_all_player_stats_df(segment=seg)

url: http://www.basketball-reference.com/players/b/brogdma01.html. ('Malcolm Brogdon', 1)
url: http://www.basketball-reference.com/players/b/brokaga01.html. ('Gary Brokaw', 1)
url: http://www.basketball-reference.com/players/b/brookpr01.html. ('Price Brookfield', 1)
url: http://www.basketball-reference.com/players/b/brookcl01.html. ('Clarence Brookins', 1)
url: http://www.basketball-reference.com/players/b/brookaa01.html. ('Aaron Brooks', 1)
url: http://www.basketball-reference.com/players/b/brookdi01.html. ('Dillon Brooks', 1)
url: http://www.basketball-reference.com/players/b/brookke01.html. ('Kevin Brooks', 1)
url: http://www.basketball-reference.com/players/b/brookma01.html. ('MarShon Brooks', 1)
url: http://www.basketball-reference.com/players/b/brookmi01.html. ('Michael Brooks', 1)
url: http://www.basketball-reference.com/players/b/brooksc01.html. ('Scott Brooks', 1)
url: http://www.basketball-reference.com/players/b/brownan01.html. ('Andre Brown', 1)
url: http://www.basketball-r

url: http://www.basketball-reference.com/players/b/burnsji01.html. ('Jim Burns', 1)
url: http://www.basketball-reference.com/players/b/burresc01.html. ('Scott Burrell', 1)
url: http://www.basketball-reference.com/players/b/burriar01.html. ('Art Burris', 1)
url: http://www.basketball-reference.com/players/b/burroju01.html. ('Junior Burrough', 1)
url: http://www.basketball-reference.com/players/b/burrobo01.html. ('Bob Burrow', 1)
url: http://www.basketball-reference.com/players/b/burtode02.html. ('Deonte Burton', 1)
url: http://www.basketball-reference.com/players/b/burtoed01.html. ('Ed Burton', 1)
url: http://www.basketball-reference.com/players/b/burtowi01.html. ('Willie Burton', 1)
url: http://www.basketball-reference.com/players/b/burttst01.html. ('Steve Burtt', 1)
url: http://www.basketball-reference.com/players/b/busedo01.html. ('Don Buse', 1)
url: http://www.basketball-reference.com/players/b/bustida01.html. ('David Bustion', 1)
url: http://www.basketball-reference.com/players/b/b

url: http://www.basketball-reference.com/players/c/cartewe01.html. ('Wendell Carter Jr.', 1)
url: http://www.basketball-reference.com/players/c/cartemi01.html. ('Michael Carter-Williams', 1)
url: http://www.basketball-reference.com/players/c/cartwbi01.html. ('Bill Cartwright', 1)
url: http://www.basketball-reference.com/players/c/cartyja01.html. ('Jay Carty', 1)
url: http://www.basketball-reference.com/players/c/carusal01.html. ('Alex Caruso', 1)
url: http://www.basketball-reference.com/players/c/cashco01.html. ('Cornelius Cash', 1)
url: http://www.basketball-reference.com/players/c/cashsa01.html. ('Sam Cash', 1)
url: http://www.basketball-reference.com/players/c/cassesa01.html. ('Sam Cassell', 1)
url: http://www.basketball-reference.com/players/c/casspom01.html. ('Omri Casspi', 1)
url: http://www.basketball-reference.com/players/c/catchha01.html. ('Harvey Catchings', 1)
url: http://www.basketball-reference.com/players/c/catlete01.html. ('Terry Catledge', 1)
url: http://www.basketball-

url: http://www.basketball-reference.com/players/c/cokerjo01.html. ('John Coker', 1)
url: http://www.basketball-reference.com/players/c/coleno01.html. ('Norris Cole', 1)
url: http://www.basketball-reference.com/players/c/colembe01.html. ('Ben Coleman', 1)
url: http://www.basketball-reference.com/players/c/colemde01.html. ('Derrick Coleman', 1)
url: http://www.basketball-reference.com/players/c/colemec01.html. ('E.C. Coleman', 1)
url: http://www.basketball-reference.com/players/c/colemja01.html. ('Jack Coleman', 1)
url: http://www.basketball-reference.com/players/c/colemno01.html. ('Norris Coleman', 1)
url: http://www.basketball-reference.com/players/c/colesbi01.html. ('Bimbo Coles', 1)
url: http://www.basketball-reference.com/players/c/collija02.html. ('Jason Collier', 1)
url: http://www.basketball-reference.com/players/c/colliar01.html. ('Art Collins', 1)
url: http://www.basketball-reference.com/players/c/collido02.html. ('Don Collins', 1)
url: http://www.basketball-reference.com/play

In [11]:
next_300.to_sql()

Season Age   Tm   Lg    Pos   G  GS    MP   FG   FGA  \
Player          Year                                                          
Malcolm Brogdon 1     2016-17  24  MIL  NBA     SG  75  28  1982  5.3  11.5   
                2     2017-18  25  MIL  NBA     PG  48  20  1436  6.1  12.6   
                3     2018-19  26  MIL  NBA     SG  64  64  1832  7.4  14.7   
                4     2019-20  27  IND  NBA     PG  54  54  1666  7.0  16.1   
                5     2020-21  28  IND  NBA     PG  17  17   626  8.2  17.9   
...                       ...  ..  ...  ...    ...  ..  ..   ...  ...   ...   
Mardy Collins   2     2007-08  23  NYK  NBA     SG  46   8   634  3.2   9.9   
                3     2008-09  24  TOT  NBA  PG,SG  48  14   890  4.2  10.0   
                4     2008-09  24  NYK  NBA     SG   9   0    75  3.8  11.0   
                5     2008-09  24  LAC  NBA     PG  39  14   815  4.3   9.9   
                6     2009-10  25  LAC  NBA     SG  43   0   470  3.6   9.8   

                      ...   FT%  ORB  DRB  TRB  AST  STL  BLK  TOV   PF   PTS  
Player          Year  ...                                                      
Malcolm Brogdon 1     ...  .865  0.9  3.0  3.9  5.8  1.5  0.2  2.1  2.5  13.9  
                2     ...  .882  0.6  3.3  3.9  3.8  1.0  0.3  1.7  3.2  15.7  
                3     ...  .928  1.3  4.4  5.7  4.0  0.9  0.3  1.8  2.0  19.7  
                4     ...  .892  1.0  4.7  5.7  8.2  0.8  0.2  2.8  2.1  19.3  
                5     ...  .900  0.6  3.5  4.0  7.0  1.5  0.2  2.0  2.0  22.2  
...                   ...   ...  ...  ...  ...  ...  ...  ...  ...  ...   ...  
Mardy Collins   2     ...  .605  1.1  3.2  4.3  5.1  1.4  0.5  3.2  3.2   8.2  
                3     ...  .609  1.3  3.1  4.3  4.5  1.3  0.4  2.3  3.2  10.2  
                4     ...  .444  1.0  2.9  3.8  4.8  1.0  0.0  2.4  3.4   9.6  
                5     ...  .649  1.3  3.1  4.4  4.5  1.3  0.4  2.3  3.2  10.2  
                6     ...  .619  1.0  2.8  3.8  3.4  1.8  0.1  2.5  3.4   8.5  

[1940 rows x 29 columns]

In [7]:
# all_player_stats_df

In [34]:
conn = sqlite3.connect("all_players.db")
c = player_conn.cursor()

In [35]:
master = """SELECT name FROM sqlite_master WHERE type='table'"""
c.execute(master).fetchall()

[]

In [36]:
all_players = """SELECT * FROM players_per_game"""
c.execute(all_players).fetchall()[0][:10]

OperationalError: no such table: players_per_game

In [21]:
players_from_sql = pd.read_sql(all_players, player_conn)
players_from_sql.head()

,index,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,2010-11,23,TOT,NBA,PF,49,0,640,4.4,...,.731,2.3,5.8,8.0,1.7,0.8,1.0,0.9,4.5,10.9
1,1,2010-11,23,BOS,NBA,PF,28,0,241,4.2,...,.625,2.5,5.7,8.2,1.5,0.6,1.0,0.7,5.4,9.4
2,2,2010-11,23,CLE,NBA,PF,21,0,399,4.6,...,.778,2.1,5.9,7.9,1.8,1.0,0.9,1.0,4.0,11.7
3,3,2011-12,24,CLE,NBA,PF,21,1,231,3.6,...,.750,3.1,5.1,8.3,0.9,0.9,0.5,0.6,2.6,9.4
4,0,2014-15,23,UTA,NBA,C,16,0,87,3.7,...,.429,4.1,6.2,10.3,0.4,2.5,1.2,1.2,9.5,11.2


In [6]:
r = requests.get(url)
r.from_cache

AttributeError: 'Response' object has no attribute 'from_cache'

In [29]:
player_stats(res, stat, numeric=True).dtypes

Season     object
Age         int64
Tm         object
Lg         object
Pos        object
G           int64
GS          int64
MP        float64
FG        float64
FGA       float64
FG%       float64
3P        float64
3PA       float64
3P%       float64
2P        float64
2PA       float64
2P%       float64
eFG%      float64
FT        float64
FTA       float64
FT%       float64
ORB       float64
DRB       float64
TRB       float64
AST       float64
STL       float64
BLK       float64
TOV       float64
PF        float64
PTS       float64
dtype: object